In [9]:
import pandas as pd
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn.decomposition import PCA
plt.rcParams['font.sans-serif']=['SimHei'] #图中文字体设置为黑体
plt.rcParams['axes.unicode_minus']=False
from tqdm.notebook import tqdm
from sklearn.metrics import silhouette_score
import scipy
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [10]:
def read_data(file_name):
    df = pd.read_csv(file_name)
    df = df.drop(labels=['BASIC_entity_name', 'BASIC_year'], axis=1)
    fields_df = pd.read_excel('IDOU_05_事件触发_03_大小类事件映射表_v2.0.xlsx')
    rename_dict = {}
    for i, row in fields_df.iterrows():
        rename_dict[row['大类事件代码'] + '_' + row['小类事件代码']] = row['名称']
    df = df.rename(columns=rename_dict)
    if 'REVENUEAB_test' in df.columns:
        df = df.drop(labels='REVENUEAB_test', axis=1)
    return df

In [11]:
df_all = read_data('舞弊案例全市场数据2014-2021.csv')
df_part = read_data('all_samples.csv')

In [17]:
df = df_part

In [20]:
feature_importance_dict = {}
for Y_col in tqdm(df.columns):
    rf = RandomForestRegressor(random_state=0)
    X_col = [_ for _ in df.columns if _ != Y_col]
    rf.fit(df[X_col], df[Y_col])
    fm_list = rf.feature_importances_.tolist()
    fm_list.insert(df.columns.to_list().index(Y_col) ,1)
    feature_importance_dict[Y_col] = fm_list

  0%|          | 0/41 [00:00<?, ?it/s]

In [25]:
result = pd.DataFrame(feature_importance_dict)
result.index = df.columns
result+= result.transpose()
result = result/2
result.to_excel('随机森林回归结果.xlsx')